In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
df=pd.read_csv('sts_data.csv')
df.head()

,T1,T2,T3,T4,T5,T6,d',D,TK,Vs,...,lo,x mtr,y mtrs,xinCM,yinCM,Y fc,Mes_x,Mes_y,Xd,Yd
0,0.0,0.000287,0.000770,0.000692,0.001025,0.000998,0.25,0.63,39.55068,354.701017,...,0.564584,0.007040,0.564540,0.703987,56.454010,-2.445990,-1.8,-1.7,-1.503987,0.745990
1,0.0,0.000281,0.000839,0.000585,0.001089,0.000885,0.25,0.63,39.06240,354.423844,...,0.588193,0.053865,0.585721,5.386534,58.572137,-0.327863,3.0,0.7,-1.386534,1.027863
2,0.0,0.000282,0.000810,0.000705,0.001059,0.001009,0.25,0.63,39.06240,354.423844,...,0.536937,0.012700,0.536787,1.270044,53.678677,-5.221323,-1.5,-4.6,-1.770044,0.621323
3,0.0,0.000278,0.000843,0.000636,0.001089,0.000934,0.25,0.63,39.06240,354.423844,...,0.558559,0.039917,0.557131,3.991681,55.713130,-3.186870,1.8,-3.0,-1.191681,0.186870
4,0.0,0.000281,0.000872,0.000597,0.001130,0.000899,0.25,0.63,39.55068,354.701017,...,0.554155,0.059410,0.550961,5.940989,55.096106,-3.803894,3.9,-2.2,-1.040989,1.603894


In [3]:
df.T2*=1000
df.T3*=1000
df.T4*=1000
df.T5*=1000
df.T6*=1000
df.Vs/=1000
df.head()

,T1,T2,T3,T4,T5,T6,d',D,TK,Vs,...,lo,x mtr,y mtrs,xinCM,yinCM,Y fc,Mes_x,Mes_y,Xd,Yd
0,0.0,0.287,0.770,0.692,1.025,0.998,0.25,0.63,39.55068,0.354701,...,0.564584,0.007040,0.564540,0.703987,56.454010,-2.445990,-1.8,-1.7,-1.503987,0.745990
1,0.0,0.281,0.839,0.585,1.089,0.885,0.25,0.63,39.06240,0.354424,...,0.588193,0.053865,0.585721,5.386534,58.572137,-0.327863,3.0,0.7,-1.386534,1.027863
2,0.0,0.282,0.810,0.705,1.059,1.009,0.25,0.63,39.06240,0.354424,...,0.536937,0.012700,0.536787,1.270044,53.678677,-5.221323,-1.5,-4.6,-1.770044,0.621323
3,0.0,0.278,0.843,0.636,1.089,0.934,0.25,0.63,39.06240,0.354424,...,0.558559,0.039917,0.557131,3.991681,55.713130,-3.186870,1.8,-3.0,-1.191681,0.186870
4,0.0,0.281,0.872,0.597,1.130,0.899,0.25,0.63,39.55068,0.354701,...,0.554155,0.059410,0.550961,5.940989,55.096106,-3.803894,3.9,-2.2,-1.040989,1.603894


In [4]:
def fun(X,c):
    x,h,V,alpha,t_offset = list(X)
    #T1,T2,T3,T4,T5,T6 x and y co-ordinate
    x0 = [0.0,0.0,0.63,-0.63,0.63]
    y0 = [0.0,0.25,0.0,0.25,0.25]
    T=[]
    for j in range(len(x0)):
        x_k,y_k = x0[j],y0[j]
        t_i = t_offset -((1/V)*((x-x_k)*math.sin(alpha)-y_k*math.cos(alpha)))\
            +((((x-x_k)*math.cos(alpha)+y_k*math.sin(alpha))**2+h**2)**0.5)*((1/c**2)-(1/V**2))**0.5
        T.append(t_i)
    return np.asarray(T,dtype='float')
def get_equation_value():
    X =[0.01,0.2,0.6,0.0,0.0]
    print(fun(X,.3))

In [5]:
get_equation_value()

[0.57807151 0.99473817 1.88060274 2.35229749 2.29726941]


In [6]:
def derivative_matrix(X,c):
    x,h,V,alpha,t_offset = list(X)
    #T1,T2,T3,T4,T5,T6 x and y co-ordinate
    x0 = [0.0,0.0,0.63,-0.63,0.63]
    y0 = [0.0,0.25,0.0,0.25,0.25]
    derivate=[]
    for j in range(len(x0)):
        x_k,y_k = x0[j],y0[j]
        Q = ((((x-x_k)*math.cos(alpha)+y_k*math.sin(alpha))**2+h**2)**0.5)
        dt_dx = -(math.sin(alpha)/V) + ((1/Q)*((1/c**2)-(1/V**2))**0.5)\
            *((x-x_k)*math.cos(alpha)+y_k*math.sin(alpha))*math.cos(alpha)
        dt_dh = (h/Q)*((1/c**2)-(1/V**2))**0.5
        dt_dV = (1/V**2)*((x-x_k)*math.sin(alpha)-y_k*math.cos(alpha))+(Q/(V**3*((1/c**2)-(1/V**2))**0.5))
        #dt_dc = -(Q/(c**3*((1/c**2)-(1/V**2))**0.5))
        dt_dalpha = -((x-x_k)*math.cos(alpha)+y_k*math.sin(alpha))\
                *((1/V)+(1/Q)*(((1/c**2)-(1/V**2))**0.5)*((x-x_k)*math.sin(alpha)-y_k*math.cos(alpha)))
        dt_dt_offset = 1
        derivate.append([dt_dx,dt_dh,dt_dV,dt_dalpha,dt_dt_offset])
    return np.asarray(derivate,dtype='float')

In [7]:
derivative_matrix([0.01,0.2,0.6,0.0,0.0],0.3)

array([[ 0.14415748,  2.88314966,  0.32115084, -0.01666667,  1.        ],
       [ 0.14415748,  2.88314966, -0.37329361,  0.0193727 ,  1.        ],
       [-2.74734613,  0.88624069,  1.0447793 ,  1.03333333,  1.        ],
       [ 2.75534636,  0.86104574,  0.38090601, -0.37783008,  1.        ],
       [-2.74734613,  0.88624069,  0.35033486,  0.3464968 ,  1.        ]])

In [8]:
from numpy.linalg import inv
x_old_log=[]
x_new_log=[]
for j in range(10):
    if j==0:
        #x,h,V,c,alpha,t_offset = []
        X_guess = np.array([0.01,0.2,0.6,0.0,0.0],dtype='float')
        X_old = X_guess
        #print(f'X_old:{X_old}')
    print(f'X_old:{X_old}')
    x_old_log.append(X_old)
    i=0
    M_calculated=fun(X_old,df.Vs[i])
    M_calculated=M_calculated.reshape(1,5)
    print(f'M_calculated:{M_calculated}')
    M_measured = np.asarray([df.T1[i],df.T2[i],df.T4[i],df.T5[i],df.T6[i]],dtype='float')
    M_measured =M_measured.reshape(1,5)
    print("M_measured:",M_measured)
    H =derivative_matrix(X_old,df.Vs[i])
    #print(np.matmul(np.matmul(inv(np.matmul(H.T,H)),H.T),(M_measured.T-M_calculated.T)))
    X_new = X_old.reshape(1,5).T+np.matmul(np.matmul(inv(np.matmul(H.T,H)),H.T),(M_measured.T-M_calculated.T))
    #X_new=np.array([X_new[0][0],X_new[1][0],X_new[2][0],X_new[3][0],X_new[4][0],X_new[5][0]])
    X_old= X_new.reshape(1,5)[0]
    print("X_new:",X_old)
    x_new_log.append(X_old)
    print("Residual vector after each iteration:",M_measured-M_calculated)
    if j>=1:
        if(np.all(np.isclose(x_old_log[j-1],x_old_log[j]))):
            break
            
        #if(np.array_equal(x_old_log[j-1],x_old_log[j])):
            #break
    print("\n#########################################    "+str(j)+"   *******************************************\n")

X_old:[0.01 0.2  0.6  0.   0.  ]
M_calculated:[[0.45534447 0.87201114 1.4813428  1.94135485 1.89800947]]
M_measured: [[0.    0.287 0.692 1.025 0.998]]
X_new: [-0.00181104  0.5154206   0.78535595 -0.03336809 -1.24646507]
Residual vector after each iteration: [[-0.45534447 -0.58501114 -0.7893428  -0.91635485 -0.90000947]]

#########################################    0   *******************************************

X_old:[-0.00181104  0.5154206   0.78535595 -0.03336809 -1.24646507]
M_calculated:[[0.04993097 0.36832416 0.7769776  1.12543475 1.11142248]]
M_measured: [[0.    0.287 0.692 1.025 0.998]]
X_new: [ 0.00172699  0.56348936  0.8624816  -0.03969427 -1.45004331]
Residual vector after each iteration: [[-0.04993097 -0.08132416 -0.0849776  -0.10043475 -0.11342248]]

#########################################    1   *******************************************

X_old:[ 0.00172699  0.56348936  0.8624816  -0.03969427 -1.45004331]
M_calculated:[[-0.0018877   0.28789158  0.68890008  1.02416296 